In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
from keras import backend as K
import os
import pandas as pd
import cv2
import numpy as np
import sys
sys.path.insert(0, "../")
from deeparuco.impl.utils import IoU, match_rois
from deeparuco.impl.losses import weighted_loss
from tensorflow.keras.models import load_model
from ultralytics import YOLO
import matplotlib.pyplot as plt
import csv
import time
import json
from shapely.geometry import Polygon
from deeparuco.impl.utils import ordered_corners, custom_marker_from_corners
from deeparuco.impl.customTag import find_id
from tensorflow.keras.saving import register_keras_serializable
from keras.metrics import mean_absolute_error

def calculate_iou(boxA, boxB):    
    polyA = Polygon(boxA)
    polyB = Polygon(boxB)
    
    if not polyA.is_valid or not polyB.is_valid:
        return 0.0
    
    intersection_area = polyA.intersection(polyB).area
    union_area = polyA.union(polyB).area
    
    return intersection_area / union_area if union_area > 0 else 0.0

def is_bbox_in_bounds(bbox_corners, h, w):
    marker_w = bbox_corners[2][0]-bbox_corners[0][0]
    marker_h = bbox_corners[2][1]-bbox_corners[0][1]
    for x, y in bbox_corners:
        if not (-marker_w/14 <= x <= w+marker_w/14 and -marker_h/14 <= y <= h+marker_h/14):
            return False
    return True  

@register_keras_serializable(package="Custom", name="extract_border")
def extract_border(x):
    # Extracting the outer border elements in clockwise order, starting from [0, 0] and ending at [1, 0]
    top_row = x[:, 0, :]         # First row (top border)
    bottom_row = x[:, -1, :]     # Last row (bottom border)
    left_column = x[:, 1:-1, 0]  # Left column, excluding top and bottom corners
    right_column = x[:, 1:-1, -1]  # Right column, excluding top and bottom corners
    
    # Concatenate the border elements in a specific clockwise order
    outer_border = tf.concat([
        top_row,                   # From [0, 0] to [0, 13]
        right_column,              # From [1, 13] to [12, 13]
        bottom_row[:, ::-1],       # From [13, 13] to [13, 0] (reverse order)
        left_column[::-1]          # From [12, 0] to [1, 0] (reverse order)
    ], axis=1)

    return outer_border

norm = lambda x: (x - np.min(x)) / (np.max(x) - np.min(x) + 1e-9)


2025-04-18 13:26:53.957509: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-18 13:26:53.964887: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745000813.973066 2099784 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745000813.975500 2099784 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745000813.982052 2099784 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [5]:
detector = YOLO('../models/nested2_detector_n/best.pt')
regressor = load_model("../models/nested2_regressor/corner_nested.keras",
    custom_objects={"weighted_loss": weighted_loss},)
decoder = load_model("../models/inpaint/decoder.keras")
# decoder = load_model("/home/su/deeparuco/models_mine/reflection/decoder/decoder_new0.keras")

@tf.function(reduce_retracing=True)
def refine_corners(crops):
    return regressor(crops)

@tf.function(reduce_retracing=True)
def decode_markers(markers):
    return decoder(markers)

2025-04-18 13:26:56.699627: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-04-18 13:26:56.699643: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:167] env: CUDA_VISIBLE_DEVICES="-1"
2025-04-18 13:26:56.699646: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:170] CUDA_VISIBLE_DEVICES is set to -1 - this hides all GPUs from CUDA
2025-04-18 13:26:56.699648: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:178] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-04-18 13:26:56.699650: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] retrieving CUDA diagnostic information for host: MightyMeca
2025-04-18 13:26:56.699651: I external/local_xla/xla/stream_executor/cuda/cu

One image

In [6]:
im_path = '../dataset/Apr1_2025/image_722.jpg'
im = cv2.imread(im_path)
h, w = im.shape[:2]
im_size = h * w
    
detections = detector(im, verbose=False, iou=0.5, conf=0.03)[0].cpu().boxes

xyxy = [
    [
        int(max(det[0] - (0.2 * (det[2] - det[0]) + 0.5), 0)),
        int(max(det[1] - (0.2 * (det[3] - det[1]) + 0.5), 0)),
        int(min(det[2] + (0.2 * (det[2] - det[0]) + 0.5), im.shape[1] - 1)),
        int(min(det[3] + (0.2 * (det[3] - det[1]) + 0.5), im.shape[0] - 1)),
    ]
    for det in [
        [int(val) for val in det.xyxy.cpu().numpy()[0]] for det in detections
    ]
]
crops_ori = [
        cv2.resize(im[det[1] : det[3], det[0] : det[2]], (128, 128)) for det in xyxy
    ]


for i in range(len(crops_ori)):
    cv2.imwrite(f"crop_{i}.png", crops_ori[i])

crops = [norm(crop) for crop in crops_ori]

if crops:
    norm_corners = refine_corners(np.array(crops)).numpy()
    corners = [[(pred[i], pred[i + 1]) for i in range(0, 8, 2)] for pred in norm_corners]

    # Ensure corners are ordered
    o_corners = [
        ordered_corners([c[0] for c in cs], [c[1] for c in cs]) for cs in corners
    ]

    markers = []

    for crop, cs in zip(crops_ori, o_corners):
        marker = custom_marker_from_corners(crop, cs, 128)

        # Grayscale and normalize

        markers.append(norm(cv2.cvtColor(marker, cv2.COLOR_BGR2GRAY)))

    for i in range(len(markers)):
        cv2.imwrite(f"marker_{i}.png", markers[i] * 255.0)

    # Get ids from markers

    decoder_out = np.round(decode_markers(np.array(markers)).numpy())
    ids, dists = zip(*[find_id(out) for out in decoder_out])

[ WARN:0@7.797] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


In [7]:
dists

(13,)

In [8]:
eval_path = '../dataset/Apr1_test.csv'
im_dir    = '../dataset/Apr1_2025/'
crop_size = 128


start_time = time.time()
image_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')

with open(eval_path, 'a', newline='') as eval_file:
	csv_writer = csv.writer(eval_file)
	csv_writer.writerow(['image', 'id', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4'])

	# for i in os.listdir(im_dir):
	for i in range(sum(1 for f in os.listdir(im_dir) if f.lower().endswith(image_extensions))):
		im_name = f'image_{i}.jpg'
		print(im_name)
		im_path = im_dir + im_name
		name = im_name.split('.')[0]

		im = cv2.imread(im_path)
		h, w = im.shape[:2]
		im_size = h * w
		IDs = []
		bboxes = []
		reflections = []
		sizes = []
		sizes_ratio = []
			
		# Predict bbox
		detections = detector(im, verbose=False, iou=0.5, conf=0.03)[0].cpu().boxes

		xyxy = [
			[
				int(max(det[0] - (0.2 * (det[2] - det[0]) + 0.5), 0)),
				int(max(det[1] - (0.2 * (det[3] - det[1]) + 0.5), 0)),
				int(min(det[2] + (0.2 * (det[2] - det[0]) + 0.5), im.shape[1] - 1)),
				int(min(det[3] + (0.2 * (det[3] - det[1]) + 0.5), im.shape[0] - 1)),
			]
			for det in [
				[int(val) for val in det.xyxy.cpu().numpy()[0]] for det in detections
			]
		]
		crops_ori = [
				cv2.resize(im[det[1] : det[3], det[0] : det[2]], (crop_size, crop_size)) for det in xyxy
			]
		
		crops = [norm(crop) for crop in crops_ori]
		if crops:
			norm_corners = refine_corners(np.array(crops)).numpy()
			corners = [[(pred[i], pred[i + 1]) for i in range(0, 8, 2)] for pred in norm_corners]

			# Ensure corners are ordered
			o_corners = [
				ordered_corners([c[0] for c in cs], [c[1] for c in cs]) for cs in corners
			]
			
			for crop, cs in zip(crops_ori, o_corners):
				# marker = custom_marker_from_corners(crop, cs, crop_size)
				marker = norm(cv2.cvtColor(custom_marker_from_corners(crop, cs, crop_size), cv2.COLOR_BGR2GRAY))
				markers = [marker]

				x1 = float(cs[0]) * crop_size
				y1 = float(cs[1]) * crop_size
				x2 = float(cs[2]) * crop_size
				y2 = float(cs[3]) * crop_size
				x3 = float(cs[4]) * crop_size
				y3 = float(cs[5]) * crop_size
				x4 = float(cs[6]) * crop_size
				y4 = float(cs[7]) * crop_size
				
				decoder_out = np.round(decode_markers(np.array(markers)).numpy())
				id, dist = find_id(decoder_out[0])
			
				csv_writer.writerow([im_name, id, x1, y1, x2, y2, x3, y3, x4, y4])
		else:
			csv_writer.writerow([im_name, None, None, None, None, None, None, None, None, None])		
			
				

end_time = time.time()

image_0.jpg
image_1.jpg
image_2.jpg
image_3.jpg
image_4.jpg
image_5.jpg
image_6.jpg
image_7.jpg
image_8.jpg
image_9.jpg
image_10.jpg
image_11.jpg
image_12.jpg
image_13.jpg
image_14.jpg
image_15.jpg
image_16.jpg
image_17.jpg
image_18.jpg
image_19.jpg
image_20.jpg
image_21.jpg
image_22.jpg
image_23.jpg
image_24.jpg
image_25.jpg
image_26.jpg
image_27.jpg
image_28.jpg
image_29.jpg
image_30.jpg
image_31.jpg
image_32.jpg
image_33.jpg
image_34.jpg
image_35.jpg
image_36.jpg
image_37.jpg
image_38.jpg
image_39.jpg
image_40.jpg
image_41.jpg
image_42.jpg
image_43.jpg
image_44.jpg
image_45.jpg
image_46.jpg
image_47.jpg
image_48.jpg
image_49.jpg
image_50.jpg
image_51.jpg
image_52.jpg
image_53.jpg
image_54.jpg
image_55.jpg
image_56.jpg
image_57.jpg
image_58.jpg
image_59.jpg
image_60.jpg
image_61.jpg
image_62.jpg
image_63.jpg
image_64.jpg
image_65.jpg
image_66.jpg
image_67.jpg
image_68.jpg
image_69.jpg
image_70.jpg
image_71.jpg
image_72.jpg
image_73.jpg
image_74.jpg
image_75.jpg
image_76.jpg
image_77.

In [9]:
end_time - start_time

29.292294025421143

In [4]:
import pandas as pd
import os

csv_path = '../dataset/Apr1_test.csv'
im_dir    = '../dataset/Apr1_2025/'
image_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')

df = pd.read_csv(csv_path)
df_outer = df[df['id'] == 19]
df_middle = df[df['id'] == 1]
df_inner = df[df['id'] == 43]

total = sum(1 for f in os.listdir(im_dir) if f.lower().endswith(image_extensions))
outer  = len(df_outer)  / total
middle = len(df_middle) / total
inner  = len(df_inner)  / total

valid_ids = [1, 19, 43]
filtered_df = df[df['id'].isin(valid_ids)]
image_count = filtered_df['image'].nunique()

print(f'detection rate of multi-scale: {image_count / total}')
print(f'detection rate of outer: {outer}')
print(f'detection rate of middle: {middle}')
print(f'detection rate of inner: {inner}')


detection rate of multi-scale: 0.2715551974214343
detection rate of outer: 0.2119258662369057
detection rate of middle: 0.15390813859790492
detection rate of inner: 0.032232070910556


AprilTag

In [2]:
import pandas as pd
import os

csv_path = '../dataset/Apr1_apriltag.csv'
im_dir    = '../dataset/Apr1_2025/'
image_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')

df = pd.read_csv(csv_path)
df_outer = df[df['ID'] == 19]
df_middle = df[df['ID'] == 1]
df_inner = df[df['ID'] == 43]

total = sum(1 for f in os.listdir(im_dir) if f.lower().endswith(image_extensions))
outer  = len(df_outer)  / total
middle = len(df_middle) / total
inner  = len(df_inner)  / total

valid_ids = [1, 19, 43]
filtered_df = df[df['ID'].isin(valid_ids)]
image_count = filtered_df['image'].nunique()

print(f'detection rate of multi-scale: {image_count / total}')
print(f'detection rate of outer: {outer}')
print(f'detection rate of middle: {middle}')
print(f'detection rate of inner: {inner}')


detection rate of multi-scale: 0.18372280419016923
detection rate of outer: 0.08299758259468171
detection rate of middle: 0.10072522159548751
detection rate of inner: 0.0


Ground Truth

In [14]:
import json
import csv

annotation_file = '../dataset/Apr1_gt.json'
csv_path        = '../dataset/Apr1_gt.csv'
im_dir    = '../dataset/Apr1_2025/'
image_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')
total = sum(1 for f in os.listdir(im_dir) if f.lower().endswith(image_extensions))

with open(annotation_file, 'r') as j:
    data = json.load(j)
# Open the text file and the CSV file
with open(csv_path, 'a', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['image', 'ID', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4'])
    for i in range(total):
        filepath = data[i]['image']
        filename = filepath.split('-')[-1]
        # print(filename)
        if any('keypoints' in item for item in data[i]):
            print(data[i]["keypoints"])
            height, width = data[i]["keypoints"][0]["original_height"], data[i]["keypoints"][0]["original_width"]
            tl_x = tl_y = tr_x = tr_y = bl_x = bl_y = br_x = br_y = None
            tag_id = None
            for j in range(len(data[i]["keypoints"])):
                label = data[i]["keypoints"][j]["keypointlabels"]
                if "inner" in label[0]:
                    tag_id = 43
                if "middle" in label[0]:
                    tag_id = 1
                if "outer" in label[0]:
                    tag_id = 19
                if "top_left_corner" in label[0]:
                    tl_x = data[i]["keypoints"][j]['x'] / 100 * width
                    tl_y = data[i]["keypoints"][j]['y'] / 100 * width
                if "top_right_corner" in label[0]:
                    tr_x = data[i]["keypoints"][j]['x'] / 100 * width
                    tr_y = data[i]["keypoints"][j]['y'] / 100 * height
                if "bottom_left_corner" in label[0]:
                    bl_x = data[i]["keypoints"][j]['x'] / 100 * width
                    bl_y = data[i]["keypoints"][j]['y'] / 100 * height
                if "bottom_right_corner" in label[0]:
                    br_x = data[i]["keypoints"][j]['x'] / 100 * width
                    br_y = data[i]["keypoints"][j]['y'] / 100 * height
            csv_writer.writerow([filename, tag_id, tl_x, tl_y, tr_x, tr_y, br_x, br_y, bl_x, bl_y])
        

[{'x': 83.82777111175747, 'y': 10.89638089429535, 'width': 0.23088023088023088, 'keypointlabels': ['inner_bottom_left_corner'], 'original_width': 640, 'original_height': 512}, {'x': 91.38808073089758, 'y': 11.237622360968881, 'width': 0.23088023088023088, 'keypointlabels': ['inner_bottom_right_corner'], 'original_width': 640, 'original_height': 512}, {'x': 81.91972081407948, 'y': 2.0324970402706177, 'width': 0.23088023088023088, 'keypointlabels': ['inner_top_left_corner'], 'original_width': 640, 'original_height': 512}, {'x': 89.04147820545155, 'y': 2.250472917323109, 'width': 0.23088023088023088, 'keypointlabels': ['inner_top_right_corner'], 'original_width': 640, 'original_height': 512}]
[{'x': 84.72540266725281, 'y': 13.507274490091929, 'width': 0.23088023088023088, 'keypointlabels': ['inner_bottom_left_corner'], 'original_width': 640, 'original_height': 512}, {'x': 91.39043176039475, 'y': 13.913223136499964, 'width': 0.23088023088023088, 'keypointlabels': ['inner_bottom_right_corne

In [13]:
import pandas as pd


csv_path_model = '../dataset/Apr1_test.csv'
csv_path_april = '../dataset/Apr1_apriltag.csv'
csv_path_gt = '../dataset/Apr1_gt.csv'

dfgt = pd.read_csv(csv_path_gt)
dfgt_outer = dfgt[dfgt['ID'] == 19]
dfgt_middle = dfgt[dfgt['ID'] == 1]
dfgt_inner = dfgt[dfgt['ID'] == 43]

dfm = pd.read_csv(csv_path_model)
dfm_outer = dfm[dfm['id'] == 19]
dfm_middle = dfm[dfm['id'] == 1]
dfm_inner = dfm[dfm['id'] == 43]

dfa = pd.read_csv(csv_path_april)
dfa_outer = dfa[dfa['ID'] == 19]
dfa_middle = dfa[dfa['ID'] == 1]
dfa_inner = dfa[dfa['ID'] == 43]

m_outer  = dfm_outer['image'].isin(dfgt_outer['image']).sum()  / len(dfgt_outer)
m_middle = dfm_middle['image'].isin(dfgt_middle['image']).sum() / len(dfgt_middle)
m_inner  = dfm_inner['image'].isin(dfgt_inner['image']).sum()  / len(dfgt_inner)

a_outer  = dfa_outer['image'].isin(dfgt_outer['image']).sum()  / len(dfgt_outer)
a_middle = dfa_middle['image'].isin(dfgt_middle['image']).sum() / len(dfgt_middle)
a_inner  = dfa_inner['image'].isin(dfgt_inner['image']).sum()  / len(dfgt_inner)

valid_ids = [1, 19, 43]
filtered_gt = dfgt[dfgt['ID'].isin(valid_ids)]
total_gt = filtered_gt['image'].nunique()

filtered_m = dfm[dfm['id'].isin(valid_ids)]
total_m = filtered_m['image'].nunique()

filtered_a = dfa[dfa['ID'].isin(valid_ids)]
total_a = filtered_a['image'].nunique()

print(f'detection rate (model) of multi-scale: {total_m / total_gt}')
print(f'detection rate (model) of outer: {m_outer}')
print(f'detection rate (model) of middle: {m_middle}')
print(f'detection rate (model) of inner: {m_inner}')

print(f'detection rate (apriltag) of multi-scale: {total_a / total_gt}')
print(f'detection rate (apriltag) of outer: {a_outer}')
print(f'detection rate (apriltag) of middle: {a_middle}')
print(f'detection rate (apriltag) of inner: {a_inner}')

detection rate (model) of multi-scale: 0.506006006006006
detection rate (model) of outer: 0.3704545454545455
detection rate (model) of middle: 0.6386554621848739
detection rate (model) of inner: 0.07476635514018691
detection rate (apriltag) of multi-scale: 0.34234234234234234
detection rate (apriltag) of outer: 0.1431818181818182
detection rate (apriltag) of middle: 0.6638655462184874
detection rate (apriltag) of inner: 0.0


In [15]:
TP = dfm_middle['image'].isin(dfgt_middle['image']).sum()
FP = (~dfm_middle['image'].isin(dfgt_middle['image'])).sum()

precision_middle = TP/(TP+FP)
precision_middle

0.39790575916230364

In [16]:
TP

76

In [17]:
FP

115

In [2]:
dfm_middle_count = dfm_middle[dfm_middle['image'].isin(dfgt_middle['image'])]
# dfm_middle_count = dfm_middle[dfm_middle['image'].isin(dfgt_middle['image'])]
dfm_middle_count

,image,id,x1,y1,x2,y2,x3,y3,x4,y4
